In [30]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from tqdm import tqdm
import util

In [31]:
DAYTYPE = ['토요일', '일요일', '공휴일', '평일']
PATH_STATION_SCHEDULE_REFINE_DAYTYPE = [f"../trimmed_data/stationSchedule{day}.json" for day in DAYTYPE]
PATH_STATION_INTERVAL_DAYTYPE= [f"../trimmed_data/stationSchedule{day}-SEPERATED.json" for day in DAYTYPE]

# `03-station-schedule-seperation.ipynb`
02 에서 만든 데이터로 쪼게기.

## 1. 데이터 불러오기

In [32]:
path = util.PATH_STATION_INFO_REFINE
sName_sCode = {}
for key,val in pd.read_json(path).iloc[:,[0,1]].transpose().to_dict().items():
    # for k,v in key.items():
    sName_sCode[val["StationName"]]=val["StationCode"]

# readme.md 참조.
sName_sCode = {key.replace("·","."):val for key,val in sName_sCode.items()}
sName_sCode = {key.replace("국제금융센터","국제금융"):val for key,val in sName_sCode.items()}
util.savePickle(sName_sCode,"../trimmed_data/stationNameCode.pickel")
sName_sCode.items()

dict_items([('다대포해수욕장', 95), ('다대포항', 96), ('낫개', 97), ('신장림', 98), ('장림', 99), ('동매', 100), ('신평', 101), ('하단', 102), ('당리', 103), ('사하', 104), ('괴정', 105), ('대티', 106), ('서대신', 107), ('동대신', 108), ('토성', 109), ('자갈치', 110), ('남포', 111), ('중앙', 112), ('부산역', 113), ('초량', 114), ('부산진', 115), ('좌천', 116), ('범일', 117), ('범내골', 118), ('서면', 219), ('부전', 120), ('양정', 121), ('시청', 122), ('연산', 305), ('교대', 124), ('동래', 402), ('명륜', 126), ('온천장', 127), ('부산대', 128), ('장전', 129), ('구서', 130), ('두실', 131), ('남산', 132), ('범어사', 133), ('노포', 134), ('장산', 201), ('중동', 202), ('해운대', 203), ('동백', 204), ('벡스코', 205), ('센텀시티', 206), ('민락', 207), ('수영', 301), ('광안', 209), ('금련산', 210), ('남천', 211), ('경성대.부경대', 212), ('대연', 213), ('못골', 214), ('지게골', 215), ('문현', 216), ('국제금융.부산은행', 217), ('전포', 218), ('부암', 220), ('가야', 221), ('동의대', 222), ('개금', 223), ('냉정', 224), ('주례', 225), ('감전', 226), ('사상', 227), ('덕포', 228), ('모덕', 229), ('모라', 230), ('구남', 231), ('구명', 232), ('덕천', 313), ('수정', 234), ('화명', 2

In [33]:
stationNameCode = util.loadPickle("../trimmed_data/stationNameCode.pickel")

for day,path,savepath in zip(DAYTYPE,PATH_STATION_SCHEDULE_REFINE_DAYTYPE,PATH_STATION_INTERVAL_DAYTYPE):
    db = pd.read_json(path,encoding="UTF-8")
    column = db.columns.to_list()
    
    df = pd.DataFrame({c: [] for c in column})
    for i in tqdm(db.index):
        tCode, lNumber, dVect, sList, aSchedule, dSchedule = db.loc[i, :]
        for s, a, d in zip(sList, aSchedule, dSchedule):
            _unit = pd.DataFrame(
                {
                    column[0]: [int(tCode)],
                    column[2]: [int(dVect)],
                    column[1]: [int(lNumber)],
                    column[3]: [int(stationNameCode[s])],
                    column[4]: [a],
                    column[5]: [d],
                    "DayType": day,
                }
            )
            # import sys;sys.exit()
            # df.loc[len(df)] = pd.Series({'A': 6, 'B': 'f'})   # 1 minute 28.3 sec
            # df = df._append(_unit)                            # 38.5 sec
            df = pd.concat([df, _unit], ignore_index=True)      # 35.6 sec
    
    # 모든 숫자는 정수형 입니다. 타입 수정을 위해 수정합니다.
    trim_target = column[:8]
    for idx in trim_target:
        df[idx] = pd.to_numeric(df[idx],downcast="integer")
    
    with open(savepath,mode="w",encoding="UTF-8") as f:
        f.write(df.to_json(index=False))
        print(f"SAVE >>> {savepath}")

100%|██████████| 1277/1277 [00:45<00:00, 27.93it/s]


SAVE >>> ../trimmed_data/stationSchedule토요일-SEPERATED.json


100%|██████████| 278/278 [00:03<00:00, 72.25it/s]


SAVE >>> ../trimmed_data/stationSchedule일요일-SEPERATED.json


100%|██████████| 924/924 [00:34<00:00, 26.44it/s]


SAVE >>> ../trimmed_data/stationSchedule공휴일-SEPERATED.json


100%|██████████| 1354/1354 [00:46<00:00, 28.98it/s]

SAVE >>> ../trimmed_data/stationSchedule평일-SEPERATED.json


In [34]:
df.tail()

,TrainCode,LineNumber,DirectionVector,StationList,ArriveSchedule,DepartSchedule,DayType
38446,4310,4,7,410,1476,1476.0,평일
38447,4310,4,7,411,1478,1479.0,평일
38448,4310,4,7,412,1481,1481.0,평일
38449,4310,4,7,413,1482,1483.0,평일
38450,4310,4,7,414,1484,NaN,평일
